In [1]:
%load_ext rich
from ipywidgets import interact
import io
import itertools
import os
import glob
import pandas as pd


from docket_extract import *

In [2]:
ds_dir = "../../data/example_docketsheets_courtsummaries/"
fnames = list(sorted(glob.glob(os.path.join(ds_dir, "*DS_*pdf"))))
fnames

[
    '../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MC-51-CR-0016214-2020.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-05216-CR-0000076-2022.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-07104-CR-0000001-2023.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-07108-CR-0000086-2020.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-12301-CR-0000168-2017.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-19310-CR-0000176-2011.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-26304-CR-0000183-2006.pdf'
]

In [3]:
sectioned = {fname: extract_sections(extract_text_from_pdf(fname)) for fname in fnames}


In [4]:
all_keys = set(itertools.chain(*[list(d.keys()) for d in sectioned.values()]))
key_count = {k: sum([k in d for d in sectioned.values()]) for k in all_keys}
list(sorted(key_count.items(), key=lambda x: x[1], reverse=True))

[
    ('', 10),
    ('DEFENDANT INFORMATION', 10),
    ('CASE INFORMATION', 10),
    ('CHARGES', 10),
    ('CASE PARTICIPANTS', 10),
    ('STATUS INFORMATION', 10),
    ('CALENDAR EVENTS', 10),
    ('DOCKET ENTRY INFORMATION', 7),
    ('DOCKET', 7),
    ('BAIL', 7),
    ('ATTORNEY INFORMATION', 7),
    ('DISPOSITION / SENTENCING DETAILS', 7),
    ('CONFINEMENT', 4),
    ('CASE FINANCIAL INFORMATION', 3),
    ('ENTRIES', 3),
    ('BAIL INFORMATION', 3),
    ('CRIMINAL DOCKET', 3),
    ('DISPOSITION SENTENCING/PENALTIES', 3),
    ('COMMONWEALTH INFORMATION            ATTORNEY INFORMATION', 3),
    ('PAYMENT PLAN SUMMARY', 1),
    (
        'MUNICIPAL COURT OF PHILADELPHIA COUNTY                           \n                                       DOCKET',
        1
    ),
    ('WRITING', 1),
    ('RELATED CASES', 1),
    (
        'COURT OF COMMON PLEAS OF FOREST COUNTY                            \n                                       DOCKET',
        1
    ),
    (
        'COURT OF COMMON PLEAS OF BEDFORD COUNTY                            \n                                       DOCKET',
        1
    )
]

In [5]:

replacements = {
    "DISPOSITION SENTENCING/PENALTIES": "DISPOSITION / SENTENCING DETAILS"
    }
remove_keys = ["", "PAYMENT PLAN SUMMARY"]
for fname, sdict in sectioned.items():
    for replace_source, replace_target in replacements.items():
        if replace_source in sdict:
            sdict[replace_target] = sdict.pop(replace_source)
    for k in remove_keys:
        if k in sdict:
            sdict.pop(k)
all_keys = set(itertools.chain(*[list(d.keys()) for d in sectioned.values()]))
key_count = {k: sum([k in d for d in sectioned.values()]) for k in all_keys}
list(sorted(key_count.items(), key=lambda x: x[1], reverse=True))

[
    ('DEFENDANT INFORMATION', 10),
    ('CASE INFORMATION', 10),
    ('CHARGES', 10),
    ('CASE PARTICIPANTS', 10),
    ('STATUS INFORMATION', 10),
    ('CALENDAR EVENTS', 10),
    ('DISPOSITION / SENTENCING DETAILS', 10),
    ('DOCKET ENTRY INFORMATION', 7),
    ('DOCKET', 7),
    ('BAIL', 7),
    ('ATTORNEY INFORMATION', 7),
    ('CONFINEMENT', 4),
    ('CASE FINANCIAL INFORMATION', 3),
    ('ENTRIES', 3),
    ('BAIL INFORMATION', 3),
    ('CRIMINAL DOCKET', 3),
    ('COMMONWEALTH INFORMATION            ATTORNEY INFORMATION', 3),
    (
        'MUNICIPAL COURT OF PHILADELPHIA COUNTY                           \n                                       DOCKET',
        1
    ),
    ('WRITING', 1),
    ('RELATED CASES', 1),
    (
        'COURT OF COMMON PLEAS OF FOREST COUNTY                            \n                                       DOCKET',
        1
    ),
    (
        'COURT OF COMMON PLEAS OF BEDFORD COUNTY                            \n                                       DOCKET',
        1
    )
]

In [6]:
[print(sectioned[fname]["DEFENDANT INFORMATION"]) for fname in fnames]

Name:         Young, Larry Lewis       Sex:         Male                        
    Date of Birth: 08/01/1951              Race:        White                       
    Address(es):                                                                    
    Home                                                                            
    Watsontown, PA 17777                                                            
    Advised of His Right to Apply for Assignment of Counsel? No                     
    Public Defender Requested by the Defendant? No                                  
    Application Provided for Appointment of Public Defender? Yes                    
    Has the Defendant Been Fingerprinted? Yes
Date Of Birth:  10/16/1950 City/State/Zip:  Roaring Spring, PA  16673
Date Of Birth:  06/14/1959 City/State/Zip:  Oil City, PA  16301
Date Of Birth:  09/28/1988 City/State/Zip:  Philadelphia, PA  19120
Name:         Fernacz, Richard         Sex:         Male                    

[None, None, None, None, None, None, None, None, None, None]

In [7]:
[extract_defendant_information(sectioned[fname]["DEFENDANT INFORMATION"]) for fname in fnames]

[
    {
        'dob': '08/01/1951',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'No',
        'defender_requested': 'No',
        'application_provided': 'Yes',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '10/16/1950',
        'race': None,
        'sex': None,
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '06/14/1959',
        'race': None,
        'sex': None,
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '09/28/1988',
        'race': None,
        'sex': None,
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '10/04/1945',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'No',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': None
    },
    {
        'dob': '05/08/1987',
        'race': 'White',
        'sex': 'Female',
        'counsel_advised': 'Yes',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '08/28/1986',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'Yes',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '05/21/1998',
        'race': 'White',
        'sex': 'Female',
        'counsel_advised': 'Yes',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '06/29/1969',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '01/24/1964',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'No',
        'defender_requested': 'Yes',
        'application_provided': 'Yes',
        'was_fingerprinted': 'No'
    }
]

In [8]:
sectioned[fnames[1]]["DEFENDANT INFORMATION"]

'Date Of Birth:  10/16/1950 City/State/Zip:  Roaring Spring, PA  16673'

In [9]:
for fname in fnames:
    print(fname)
    print(sectioned[fname]["CHARGES"]) 
    print()

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
#Charge        Grade Description                 Offense Dt. Disposition   
         175 § 3802 §§ A1* M DUI: Gen Imp/Inc of Driving Safely - 1st Off 03/19/2015 Waived for Court
         275 § 3802 §§ B* M  DUI: High Rte of Alc (Bac.10 - <.16) 1st Off 03/19/2015 Waived for Court

../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf
Seq. Orig Seq. Grade Statute  Statute Description      Offense Dt. OTN       
        1    2     M1   75 § 3802 §§ C** DUI: Highest Rte of Alc (BAC .16+) 2nd 01/23/2006 K 367492-6

../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf
Seq. Orig Seq. Grade Statute  Statute Description      Offense Dt. OTN       
        1    1     M2   18 § 3928 §§ A Unauth Use Motor/Other Vehicles 02/14/2005 H 029661-2
        3    3     M2   18 § 3921 §§ A Theft By Unlaw Taking-Movable Prop 02/14/2005 H 029661-2
        5    5     M2   18 § 3925 §§ A 

In [10]:
import re

for fname in fnames:
    if "Statute" in sectioned[fname]["CHARGES"]:
        charges_df = extract_charges_MC(sectioned[fname]["CHARGES"])
    else:
        charges_df = extract_charges_MJ(sectioned[fname]["CHARGES"])
    print(fname) 
    print(charges_df)
    print()

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
   #             Charge Grade                                   Description  \
0  1   75 § 3802 §§ A1*     M  DUI: Gen Imp/Inc of Driving Safely - 1st Off   
1  2  75 § 3802 §§ B* M        DUI: High Rte of Alc (Bac.10 - <.16) 1st Off   

  Offense Dt.       Disposition  
0  03/19/2015  Waived for Court  
1  03/19/2015  Waived for Court  

../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf
  Seq Orig Seq Grade           Statute  \
0   1        2    M1  75 § 3802 §§ C**   

                      Statute Description Offense Dt.         OTN  
0  DUI: Highest Rte of Alc (BAC .16+) 2nd  01/23/2006  K 367492-6  

../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf
  Seq Orig Seq Grade         Statute                 Statute Description  \
0   1        1    M2  18 § 3928 §§ A     Unauth Use Motor/Other Vehicles   
1   3        3    M2  18 § 3921 §§ A  Theft By Unlaw 

In [11]:
[print(sectioned[fname]["STATUS INFORMATION"]) for fname in fnames]

Case Status    Status Date Processing Status                                    
    Closed         06/04/2015 Case Transferred to Court of Common Pleas             
                   06/02/2015 Completed                                             
                   04/14/2015 Awaiting Preliminary Hearing
Case Status: Closed  Status Date Processing Status        Complaint Date: 02/09/2006
                          07/26/2006 Sentenced/Penalty Imposed                      
                          06/12/2006 Awaiting Sentencing                            
                          06/07/2006 Awaiting Plea Court                            
                          05/02/2006 Awaiting Status Conference                     
                          04/11/2006 Awaiting Formal Arraignment                    
                          03/23/2006 Awaiting Filing of Information
Case Status: Closed  Status Date Processing Status          Arrest Date: 04/14/2005
                          11

[None, None, None, None, None, None, None, None, None, None]

In [12]:


# Example usage
text = """Case Status    Closed
    Status Date Processing Status                                    
    07/30/2020 Case Transferred to Court of Common Pleas             
    07/23/2020 Completed                                             
    07/23/2020 Case Balance Due                                      
    07/23/2020 Completed                                             
    04/02/2020 Awaiting Preliminary Hearing                          
    04/02/2020 Awaiting Preliminary Arraignment"""

case_status, status_df = extract_status_information(text)
print(case_status)
print(status_df)

# Additional example usage
text = """Case Status    Status Date Processing Status                                    
    Closed         06/04/2015 Case Transferred to Court of Common Pleas             
                   06/02/2015 Completed                                             
                   04/14/2015 Awaiting Preliminary Hearing"""

case_status, status_df = extract_status_information(text)
print(case_status)
print(status_df)

Closed
  status_date                          processing_status
0  07/30/2020  Case Transferred to Court of Common Pleas
1  07/23/2020                                  Completed
2  07/23/2020                           Case Balance Due
3  07/23/2020                                  Completed
4  04/02/2020               Awaiting Preliminary Hearing
5  04/02/2020           Awaiting Preliminary Arraignment
Closed
  status_date                          processing_status
0  06/04/2015  Case Transferred to Court of Common Pleas
1  06/02/2015                                  Completed
2  04/14/2015               Awaiting Preliminary Hearing


In [13]:
for fname in fnames:
    print(fname)
    print(extract_status_information(sectioned[fname]["STATUS INFORMATION"]))
    print()

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
('Closed',   status_date                          processing_status
0  06/04/2015  Case Transferred to Court of Common Pleas
1  06/02/2015                                  Completed
2  04/14/2015               Awaiting Preliminary Hearing)

../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf
('Closed',   status_date                     processing_status
0  02/09/2006  07/26/2006 Sentenced/Penalty Imposed
1  06/12/2006                   Awaiting Sentencing
2  06/07/2006                   Awaiting Plea Court
3  05/02/2006            Awaiting Status Conference
4  04/11/2006           Awaiting Formal Arraignment
5  03/23/2006        Awaiting Filing of Information)

../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf
('Closed',    status_date                  processing_status
0   04/14/2005               11/14/2005 Completed
1   09/22/2005  Awaiting Appellate Cour

In [14]:
key = "CALENDAR EVENTS"
[print(sectioned[fname][key]) for fname in fnames]

Case Calendar    Schedule                                        Schedule       
    Event Type       Start Date Start Time Room    Judge Name        Status         
    Preliminary Hearing 05/12/2015  3:00 pm        Magisterial District Judge  Continued
                                                   Jeffrey L. Mensch
Case Calendar  Schedule  Start Room       Judge Name          Schedule        
      Event Type    Start Date Time                                 Status          
      Plea Court    06/12/2006  9:00 am Courtroom 1                 Scheduled       
      Parole Hearing 11/06/2006  3:00 pm Courtroom 1                Scheduled
Case Calendar  Schedule  Start Room       Judge Name          Schedule        
      Event Type    Start Date Time                                 Status          
      Formal Arraignment 06/03/2005  9:00 am                        Scheduled
Case Calendar  Schedule  Start Room       Judge Name          Schedule        
      Event Type    Start Da

[None, None, None, None, None, None, None, None, None, None]

In [15]:



# Example usage
text = """Case Calendar  Schedule  Start Room       Judge Name          Schedule        
      Event Type    Start Date Time                                 Status          
      Preliminary   08/27/2020  5:58 pm B08                         Scheduled       
      Arraignment                                                                   
      Status        10/14/2020 10:00 am 404     Judge Patrick F. Dugan Scheduled    
      Status        12/02/2020  9:00 am 200                         Cancelled       
      Trial         09/16/2021 11:30 am 606     Judge David H. Conroy Continued     
      Trial         12/10/2021 10:00 am 405     Judge David H. Conroy Scheduled"""

events_df = extract_calendar_events(text)
print(events_df)

        start_datetime               event_type room                   judge  \
0  08/27/2020  5:58 pm  Preliminary Arraignment  B08                           
1  10/14/2020 10:00 am                   Status  404  Judge Patrick F. Dugan   
2  12/02/2020  9:00 am                     None  200                           
3  09/16/2021 11:30 am                    Trial  606   Judge David H. Conroy   
4  12/10/2021 10:00 am                    Trial  405   Judge David H. Conroy   

      status  
0  Scheduled  
1  Scheduled  
2  Cancelled  
3  Continued  
4  Scheduled  


In [16]:
for fname in fnames:
    print(fname)
    print(extract_calendar_events(sectioned[fname][key]))
    print()

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
        start_datetime           event_type  room  \
0  05/12/2015  3:00 pm  Preliminary Hearing  None   

                                          judge     status  
0  Magisterial District Judge Jeffrey L. Mensch  Continued  

../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf
        start_datetime      event_type room                judge     status
0  06/12/2006  9:00 am      Plea Court    1  Courtroom Courtroom  Scheduled
1  11/06/2006  3:00 pm  Parole Hearing    1            Courtroom  Scheduled

../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf
        start_datetime          event_type  room judge     status
0  06/03/2005  9:00 am  Formal Arraignment  None        Scheduled

../../data/example_docketsheets_courtsummaries/DS_MC-51-CR-0016214-2020.pdf
        start_datetime               event_type room                   judge  \
0  08/27/2020  5:58 

In [17]:
key = "CASE PARTICIPANTS"
[print(sectioned[fname][key]) for fname in fnames]

Participant Type         Participant Name                                       
    Arresting Officer        Andres, Maxwell W.                                     
    Defendant                Young, Larry Lewis
Participant Type        Name                                                   
     Defendant               Claar, Patricia Ann
Participant Type        Name                                                   
     Defendant               Loll, Daniel Mark
Participant Type        Name                                                   
     Defendant               Johnson, Edward
Participant Type         Participant Name                                       
    Defendant                Fernacz, Richard                                       
    Arresting Officer        Stewart, William R.
Participant Type         Participant Name                                       
    Defendant                Carty, Danielle M.                                     
    Arresting Officer   

[None, None, None, None, None, None, None, None, None, None]

In [ ]:
defendants = {}
for fname in fnames:
    print(fname)
    participants = extract_case_participants(sectioned[fname][key])
    print(participants)
    defend_row = participants[participants["role"].str.contains("Defendant")]["name"]
    if len(defend_row) > 0:
        defendants[fname] = defend_row.values[0]
    else:
        defendants[fname] = ""
    print()

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
        role                               name
0  Arresting  Officer        Andres, Maxwell W.
1  Defendant                 Young, Larry Lewis

../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf
        role                 name
0  Defendant  Claar, Patricia Ann

../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf
        role               name
0  Defendant  Loll, Daniel Mark

../../data/example_docketsheets_courtsummaries/DS_MC-51-CR-0016214-2020.pdf
        role             name
0  Defendant  Johnson, Edward

../../data/example_docketsheets_courtsummaries/DS_MJ-05216-CR-0000076-2022.pdf
        role                                name
0  Defendant                    Fernacz, Richard
1  Arresting  Officer        Stewart, William R.

../../data/example_docketsheets_courtsummaries/DS_MJ-07104-CR-0000001-2023.pdf
        role                              name

In [19]:
key = "DOCKET ENTRY INFORMATION"
[print(sectioned[fname][key]) for fname in fnames if key in sectioned[fname]]

Filed Date Entry                Filer                Applies To                 
    06/02/2015 Waiver of Preliminary Hearing Larry Lewis Young Larry Lewis Young, Defendant
    06/02/2015 Waived for Court     Magisterial District Judge Jeffrey L.  Larry Lewis Young, Defendant
                                    Mensch                                          
    05/05/2015 First Class Summons Accepted Magisterial District Court 17-3-02 Larry Lewis Young, Defendant
    04/20/2015 Certified Summons Accepted Magisterial District Court 17-3-02 Larry Lewis Young, Defendant
    04/14/2015 First Class Fingerprint Order Issued Magisterial District Court 17-3-02 Larry Lewis Young, Defendant
    04/14/2015 Fingerprint Order Issued Magisterial District Court 17-3-02 Larry Lewis Young, Defendant
    04/14/2015 Summons Issued       Magisterial District Court 17-3-02 Larry Lewis Young, Defendant
    04/14/2015 Certified Summons Issued Magisterial District Court 17-3-02 Larry Lewis Young, Defendant


[None, None, None, None, None, None, None]

In [20]:
for fname in fnames:
    print(fname)
    if key in sectioned[fname]:
        docket_entry = extract_docket_entry(sectioned[fname][key], defendants[fname])
        print(docket_entry)

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
         date                          entry                    applies_to  \
0  06/02/2015  Waiver of Preliminary Hearing  Larry Lewis Young, Defendant   
1  06/02/2015               Waived for Court  Larry Lewis Young, Defendant   
2  05/05/2015   First Class Summons Accepted  Larry Lewis Young, Defendant   
3  04/20/2015               Summons Accepted  Larry Lewis Young, Defendant   
4  04/14/2015                                 Larry Lewis Young, Defendant   
5  04/14/2015                                 Larry Lewis Young, Defendant   
6  04/14/2015                 Summons Issued  Larry Lewis Young, Defendant   
7  04/14/2015                 Summons Issued  Larry Lewis Young, Defendant   
8  04/14/2015       Criminal Complaint Filed                                 
9  04/14/2015     First Class Summons Issued  Larry Lewis Young, Defendant   

                                               filer  
0     

In [21]:
key = "ATTORNEY INFORMATION"
[print(sectioned[fname][key] + "\n***\n") for fname in fnames if key in sectioned[fname]]

District Attorney                    Assistant District Attorney                
    Name:  David Peter Johnson, Esq.     Name:  Jeffrey J. Crossland, Esq.          
    Representing:  Commonwealth of Pennsylvania Representing:  Commonwealth of Pennsylvania
    Counsel Status:  Active              Counsel Status:  Active                    
    Supreme Court No.:  041281           Supreme Court No.:  047578
***

Private
***

Private                              District Attorney                          
    Name:  Keith J. Bidlingmaier, Esq.   Name:  Bucks County District Attorney's Office
    Representing:  Carty, Danielle M.    Representing:  Commonwealth of Pennsylvania
***

Public Defender                      District Attorney                          
    Name:  Mary Kathryn McAloon, Esq.    Name:  Bucks County District Attorney's Office
    Representing:  Mathiak, Michael Everett Representing:  Commonwealth of Pennsylvania
    Counsel Status:  Active              Counsel Status

[None, None, None, None, None, None, None]

In [22]:
for fname in fnames:
    if key in sectioned[fname]:
        print(fname)
        print(sectioned[fname][key])
        print()
        attornies = extract_attorney_information(sectioned[fname][key])
        print(attornies)
        print()
        print("***")
        print()
        

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
District Attorney                    Assistant District Attorney                
    Name:  David Peter Johnson, Esq.     Name:  Jeffrey J. Crossland, Esq.          
    Representing:  Commonwealth of Pennsylvania Representing:  Commonwealth of Pennsylvania
    Counsel Status:  Active              Counsel Status:  Active                    
    Supreme Court No.:  041281           Supreme Court No.:  047578

                         title                        name
0            District Attorney   David Peter Johnson, Esq.
1  Assistant District Attorney  Jeffrey J. Crossland, Esq.

***

../../data/example_docketsheets_courtsummaries/DS_MJ-05216-CR-0000076-2022.pdf
Private

     title name
0  Private     

***

../../data/example_docketsheets_courtsummaries/DS_MJ-07104-CR-0000001-2023.pdf
Private                              District Attorney                          
    Name:  Keith J. Bidlingmaier, Esq. 

In [23]:
key = "BAIL"
[print(sectioned[fname][key] + "\n***\n") for fname in fnames if key in sectioned[fname]]

Bail Set:                                                        Nebbia Status: None
    Bail Action Type Bail Action Date Bail Type Originating Court Percentage Amount 
    Set             06/02/2015  Nonmonetary   Magisterial District Court     $0.00
***

Bail Set:                                                        Nebbia Status: None
    Bail Action Type Bail Action Date Bail Type Originating Court Percentage Amount 
    Set             03/25/2022  Nonmonetary   Magisterial District Court     $0.00
***

Bail Set:                                                        Nebbia Status: None
    Bail Action Type Bail Action Date Bail Type Originating Court Percentage Amount 
    Set             01/01/2023  Unsecured     Magisterial District Court  $10,000.00
    Change Bail Type 01/02/2023 Monetary      Magisterial District Court  10.00% $75,000.00
***

Bail Set:                                                        Nebbia Status: None
    Bail Action Type Bail Action Date Bail Type

[None, None, None, None, None, None, None]

In [24]:
for fname in fnames:
    if key in sectioned[fname]:
        print(fname)
        print(sectioned[fname][key])
        print()
        bail_info = extract_bail(sectioned[fname][key])
        print(bail_info)
        print()
        print("***")
        print()

../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf
Bail Set:                                                        Nebbia Status: None
    Bail Action Type Bail Action Date Bail Type Originating Court Percentage Amount 
    Set             06/02/2015  Nonmonetary   Magisterial District Court     $0.00

{'nebbia_status': 'None', 'actions': [{'action_type': 'Set', 'action_date': '06/02/2015', 'bail_type': 'Nonmonetary', 'originating_court': 'Magisterial District Court', 'amount': '$0.00'}]}

***

../../data/example_docketsheets_courtsummaries/DS_MJ-05216-CR-0000076-2022.pdf
Bail Set:                                                        Nebbia Status: None
    Bail Action Type Bail Action Date Bail Type Originating Court Percentage Amount 
    Set             03/25/2022  Nonmonetary   Magisterial District Court     $0.00

{'nebbia_status': 'None', 'actions': [{'action_type': 'Set', 'action_date': '03/25/2022', 'bail_type': 'Nonmonetary', 'originating_court'